In [ ]:
%load_ext autoreload
%autoreload 2

# Unbiased ECMWF

Here we propose a small model which is a debiased ECMWF forecast according to the data we have.
The plan is
* Compute the bias between the ECMWF model and the observations
* Make a debiased model
* Turn this model into a probabilistic forecast
For this notebook we want to do it on precipitation and temperature, for weeks 1-2, 3-4, and 5-6.

In [ ]:
import dask
import dask.array as da
import dask.distributed
import datetime
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import pathlib
import scipy.stats
import typing
import xarray as xr
import xskillscore as xs

In [ ]:
from crims2s.dask import create_dask_cluster
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OBSERVATIONS = '***BASEDIR***/processed/training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'

## Boost dask cluster

In [ ]:
cluster = create_dask_cluster()
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

## Generic Functions

In [ ]:
def extract_train_validation_from_lead_time(xr_data) -> typing.Tuple:
    xr_data_sub_train = xr_data.sel(forecast_year=slice(None, 2018))
    xr_data_sub_val = xr_data.sel(forecast_year=slice(2019, None))
    
    return xr_data_sub_train, xr_data_sub_val

In [ ]:
def compute_and_correct_bias(data_center_train, data_center_val, obs_train):
    
    bias = (obs_train - data_center_train).mean(dim=['lead_time', 'forecast_year'])
    corrected_bias = data_center_val + bias
    
    return bias, corrected_bias

In [ ]:
def add_biweekly_dim(dataset):
    weeklys = []
    for s in [slice('0D', '13D'), slice('14D', '27D'), slice('28D', '41D')]:
        weekly_forecast = dataset.sel(lead_time=s)

        first_lead = pd.to_timedelta(weekly_forecast.lead_time[0].item())

        weekly_forecast = weekly_forecast.expand_dims(dim='biweekly_forecast').assign_coords(biweekly_forecast=[first_lead])
        weekly_forecast = weekly_forecast.assign_coords(lead_time=(weekly_forecast.lead_time - first_lead))
        weeklys.append(weekly_forecast)
        
    return xr.concat(weeklys, dim='biweekly_forecast').transpose('forecast_year', 'forecast_dayofyear', 'biweekly_forecast', ...)

## Read data

### ECMWF Temperature

In [ ]:
CENTER = 'ecmwf'
FIELD = 't2m'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
input_files_t2m = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
input_files_t2m[:10]

In [ ]:
ecmwf_t2m_raw = xr.open_mfdataset(input_files_t2m, preprocess=fix_dataset_dims)
ecmwf_t2m_raw = ecmwf_t2m_raw.assign_coords(lead_time=ecmwf_t2m_raw.lead_time - ecmwf_t2m_raw.lead_time[0])
# Fix the lead times by starting them at 0. To be validated with the organizers.

In [ ]:
ecmwf_t2m = add_biweekly_dim(ecmwf_t2m_raw)

In [ ]:
ecmwf_t2m

### Observations

In [ ]:
obs_path = pathlib.Path(OBSERVATIONS)
obs_files = [f for f in obs_path.iterdir() if 't2m' in f.stem]

In [ ]:
obs_files[:4]

In [ ]:
obs_t2m_raw = xr.open_mfdataset(obs_files)
obs_t2m_raw = obs_t2m_raw.assign_coords(lead_time=obs_t2m_raw.lead_time - obs_t2m_raw.lead_time[0])

In [ ]:
obs_t2m = add_biweekly_dim(obs_t2m_raw)

In [ ]:
obs_t2m

In [ ]:
ecmwf_t2m_train, ecmwf_t2m_val = extract_train_validation_from_lead_time(ecmwf_t2m)

In [ ]:
obs_t2m_train, obs_t2m_val = extract_train_validation_from_lead_time(obs_t2m)

In [ ]:
ecmwf_t2m_train

In [ ]:
obs_t2m_train

## Debiasing

### Compute bias using training data

In [ ]:
ecmwf_t2m_bias = (obs_t2m_train - ecmwf_t2m_train).mean(dim=['lead_time', 'forecast_year'])

In [ ]:
ecmwf_t2m_bias

### Bias correct ECMWF

In [ ]:
ecmwf_t2m_val_corrected = ecmwf_t2m_val + ecmwf_t2m_bias

In [ ]:
ecmwf_t2m_val_corrected

In [ ]:
ecmwf_t2m_val_corrected_comp = ecmwf_t2m_val_corrected.compute()

## Turn into probabilistic forecast

### Get thresholds from train observations

In [ ]:
obs_t2m_train_thresholds = obs_t2m_train.chunk({'forecast_year': -1}).quantile([0.33, 0.67], dim=['lead_time', 'forecast_year'])

In [ ]:
obs_t2m_train_thresholds

In [ ]:
obs_t2m_train_thresholds_comp = obs_t2m_train_thresholds.compute()

### Compute p of thresholds according to the model

There are two ways to do this. 
We can either count the amount of members that are whithin each category.
Or compute a distribution of all the members of the model, and then compute the value of the CDF for each threshold.

Here we do it using the distribution method.

#### Compute a distribution of the members of the model

In [ ]:
ecmwf_t2m_val_corrected_mean = ecmwf_t2m_val_corrected_comp.mean(dim=['realization', 'lead_time'])
ecmwf_t2m_val_corrected_std = ecmwf_t2m_val_corrected_comp.std(dim=['realization', 'lead_time'])

#### Compute the value of the CDF for each threshold

In [ ]:
ecmwf_t2m_val_corrected_mean

In [ ]:
ecmwf_t2m_val_corrected_mean.isel(biweekly_forecast=1, forecast_dayofyear=25).t2m.plot()

In [ ]:
obs_t2m_train_thresholds_comp

In [ ]:
def make_probabilistic(forecast, thresholds):   
    loc = forecast.mean(dim=['realization', 'lead_time']).compute().t2m
    scale = forecast.std(dim=['realization', 'lead_time']).compute().t2m
    
    cdfs = xr.apply_ufunc(scipy.stats.norm.cdf, thresholds.t2m, dask='allowed', kwargs={'loc': loc, 'scale': scale})
    
    below = cdfs.isel(quantile=0).drop_vars('quantile')
    normal = (cdfs.isel(quantile=1) - cdfs.isel(quantile=0))
    above = xr.ones_like(normal) - cdfs.isel(quantile=1).drop_vars('quantile')
    
    return xr.Dataset({'t2m': xr.concat([below, normal, above], 'category').assign_coords(category=['below normal', 'near normal', 'above normal'])})

In [ ]:
val_probabilistic_forecast = make_probabilistic(ecmwf_t2m_val_corrected_comp, obs_t2m_train_thresholds_comp)

In [ ]:
val_probabilistic_forecast = val_probabilistic_forecast.expand_dims('forecast_year').assign_coords(forecast_year=ecmwf_t2m_val_corrected_comp.forecast_year)

In [ ]:
#val_probabilistic_forecast = val_probabilistic_forecast.assign_coords(valid_time=ecmwf_t2m_val_corrected_comp.valid_time)

In [ ]:
val_probabilistic_forecast.biweekly_forecast.data

In [ ]:
val_probabilistic_forecast = val_probabilistic_forecast.rename_dims({'biweekly_forecast': 'lead_time'}).assign_coords(lead_time=val_probabilistic_forecast.biweekly_forecast.data)

In [ ]:
val_probabilistic_forecast

In [ ]:
val_probabilistic_forecast.to_netcdf('***BASEDIR***/test_t2m_forecast.nc')

In [ ]:
val_probabilistic_forecast.isel(category=2, forecast_dayofyear=40, biweekly_forecast=2).plot()

In [ ]:
val_probabilistic_forecast.isel(category=1, forecast_dayofyear=40, biweekly_forecast=0).plot()

### Sanity check

In [ ]:
val_probabilistic_forecast.sum(dim='category').isel(forecast_dayofyear=0, biweekly_forecast=2).plot()

## Make submission file out of it

In [ ]:
val_probabilistic_forecast

In [ ]:
val_probabilistic_forecast_unfixed =val_probabilistic_forecast.stack(forecast_time=['forecast_year', 'forecast_dayofyear'])

In [ ]:
forecast_times = []
for f in val_probabilistic_forecast_unfixed.forecast_time:
    year, dayofyear = f.data.item()
    year = pd.to_datetime(f'{year}-01-01')
    dayofyear = pd.Timedelta(dayofyear - 1, 'D')
    forecast_times.append(year + dayofyear)

In [ ]:
forecast_time = xr.DataArray(forecast_times, dims='forecast_time')
val_probabilistic_forecast_unfixed.assign_coords(forecast_time=forecast_time).to_netcdf('***BASEDIR***/test_t2m_forecast.nc')

In [ ]:
t2m = xr.open_dataset('***BASEDIR***/test_t2m_forecast.nc')
tp = xr.open_dataset('***BASEDIR***/test_tp_forecast.nc')

In [ ]:
big = xr.merge([t2m, tp])

In [ ]:
big.to_netcdf('***BASEDIR***/test_both_forecast.nc')